
## Student Name : Yogita Wankhede
## Project Title : Home Loan Approval 

## **1.Bussiness Understanding:**
 #### 1. Problem Statement : The main problem we are addressing is that we don't know who are applicable for home loan.So to solve this problem we automate the loan eligibility process (real-time) based on customer detail provided while filling out the online application form. These details are Gender, Marital Status, Education, Number of Dependents, Income, Loan Amount, Credit History and others.
 #### 2.In our data "LOAN STATUS" column is our target variable and all other remening columns like 'Gender','Married','Dependents','Education','Self Employed','Applicant Income','Coapplicant Income','Loan amount','Loan Amount Term','Credit History' and 'property Area' are features(Independent variables) to be used to predict the result. 
 
 #### 3.Our task is of Classification.So for model building we use SVM model and logistic Regression model and Random Forest Model.

## **2.Data Understanding**

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#1.load data set  
df= pd.read_csv('/kaggle/input/home-loan-approval/loan_sanction_train.csv')

#2.show top 5 rows
df.head()

#### Observation: We load the data set (Home Loan approval) and show its top 5 rowas.

In [ ]:
# checking number of rows and columns 
df.shape

#### Observation: We have 614 rows and 13 columns in our data set.

In [ ]:
df.columns

In [ ]:
df.dtypes

#### Observation : we have int,float and object types of data in our data set

In [ ]:
df.info()

In [ ]:
df.drop('Loan_ID', axis=1, inplace= True)
df.columns

In [ ]:
#checking data types of each column
df.info()

In [ ]:
df['ApplicantIncome'].count()

In [ ]:
#generate summary statistics for each column
df.describe()

In [ ]:
df['LoanAmount'].min()

In [ ]:
df['LoanAmount'].max()

#### Observations:Loan Amount is positivly correlated with Applicant income , Coapplicant income , loan amount terms and loan amount is negativly correlated with Credit history. 


In [ ]:
df.isnull().sum()

## 3.EDA

In [ ]:
#visualizing the class distribution for target variable(our target variable is loan status)
plt.figure(figsize=(4,4))
sns.countplot(x='Loan_Status',data=df)
plt.title('Loan Status Distribution')
plt.xlabel('Loan_Status')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(4,4))
sns.countplot(x='Gender',data=df)
plt.title('Loan Status Distribution')
plt.xlabel('Loan_Status')
plt.ylabel('Count')
plt.show()

In [ ]:
# barplot for categorical variables
cat_cols = ['Gender','Married','Education','Self_Employed','Property_Area','Dependents','Credit_History','Loan_Status']
plt.figure(figsize=(14,12))
for i,col in enumerate(cat_cols,1):
    plt.subplot(4,2,i)
    sns.countplot(x=col,hue='Loan_Status', data=df)
    plt.xlabel(col)
    plt.ylabel('count')
    plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
#pairplot for numerical variables ( for finding pairwise relationship between the numerical variables)
num_cols =['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term']
sns.pairplot(df[num_cols],diag_kind='kde')
plt.suptitle('pairplot of Numerical Features',y=1.02)
plt.show()

In [ ]:
# boxplot (for outlier detection in numerical features)
plt.figure(figsize=(12,8))
for i,col in enumerate(num_cols,1):
    plt.subplot(2,3,i)
    sns.boxplot(x='Loan_Status',y=col, data = df)
    plt.title(f'Box plot of{col}by Loan Status')
    plt.xlabel('Loan Status')
    plt.ylabel(col)
plt.tight_layout()
plt.show()

In [ ]:
#scatterplot
plt.scatter(df['ApplicantIncome'],df['LoanAmount'])
plt.title("Applicant Income Vs Loan amount")
plt.xlabel("ApplicantIncome")
plt.ylabel("LoanAmount")
plt.show()


In [ ]:
#generating correlation matrix 
print('Correlation')
display(df.corr(numeric_only=True))

In [ ]:
#correlation matrics
sns.heatmap(df.corr(numeric_only = True),annot=True)
plt.show()

In [ ]:
df.info()

In [ ]:
plt.plot(df['ApplicantIncome'],df['Credit_History'])
plt.show()

In [ ]:
bar = plt.bar(df['Property_Area'],df['LoanAmount'])
plt.show()

#### Observations:
##### 1. There is high possiblity of Male to get Loan.
##### 2.Male who are married , graduate and zero depndents also have high possiblity of get the loan.
##### 3.Self Employed have less possiblity of get the loan.
##### 4.Some Outliers are detected in our data.

In [ ]:
plt.pie(x=df['Gender'])
plt.show()

In [ ]:
bar,ax= plt.subplots(figsize=(4,5)) 
ax = plt.pie(x = 'Gender', labels=counts.keys(),autopct='%.0f%%') 
plt.title("Loan Approved", fontsize=17) 
plt.show()

## 4.Data Preparation

In [ ]:
(df.isnull().sum()/(len(df)))*100

#### Observation : We have missing values in  columns Gender , Married, Dependents, Self Employed, Loan Amount, Loan amount Term and Credit history 

In [ ]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
#separating the into features(X)  and target(Y)
X = df.drop(columns=['Loan_Status'],axis=1)
Y = df['Loan_Status']

In [ ]:
print(X)
print(Y)

In [ ]:
#data spliting into train and test
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.1,random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

In [ ]:
#defining categorical and numerical features
categorical_features=['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Credit_History','Loan_Status']
numerical_features=['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term']

In [ ]:
from sklearn.pipeline import Pipeline
#preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scalar',StandardScaler())])

#preprocessing for categorical data
categorical_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown ='ignore'))])

preprocessor= ColumnTransformer(
    transformers=[
    ('num',numerical_transformer,numerical_features),
    ('cat',categorical_transformer,numerical_features)])

preprocessor 

#### **Observations** :
#### 1.Split the data into train and test.
#### 2.Define the numerical data and categorical data and then create a pipeline .

### Model Selection
#### 1.Support Vector Machine

In [ ]:
classifiers = [
    ('Logistic Regression', LogisticRegression()),
    ('Random Forest Classifier', RandomForestClassifier()),
    ('Support Vector Machine', SVC())
]

for classifier_name, classifier in classifiers:
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', classifier)
    ])
  # Perform cross-validation on the training data
    cross_val_scores = cross_val_score(model, X_train, y_train, cv=4, scoring='accuracy')
    print(classifier_name)
    print(f"Cross-Validation Scores: {cross_val_scores}")
    print(f"Mean Cross-Validation Score: {np.mean(cross_val_scores)}\n")
    

#### 

#### 1.Our Data set is of Classification so we perform SVM , Logistic Regression and Random Forest model 
#### 2.SVM's Mean CV Score = 0.68
#### 3.Logistics  Mean CV Score = 0.67 
#### 4.Random Forest  Mean CV Score = 0.67